<a href="https://colab.research.google.com/github/e-margot/FLIR80/blob/main/crop_picture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import json
import os
from torchvision.transforms import ToTensor, PILToTensor, ConvertImageDtype, Compose
import os.path
from typing import Any, Callable, Optional, Tuple, List
from torchvision.datasets.vision import VisionDataset

In [ ]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, PILToTensor
import torchvision.transforms as tt
from torch.utils.data import random_split
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torchvision.models as models
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Crop picture example 

In [ ]:
img = Image.open("/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_train/data/video-4WXE26tzNxT688hT3-frame-002055-7Ghf3TZLA3GX5fWeT.jpg")
width, height = img.size
print(width, height)

In [ ]:
display(img)

In [ ]:
x_left = 147
y_left = 499
x_right = x_left + 1652
y_right = y_left + 285

area = (x_left, y_left, x_right, y_right)
cropped_img = img.crop(area)
cropped_img.show()

In [ ]:
display(cropped_img)

In [ ]:
x_left = 147
y_left = 499
x_right = x_left + 1652
y_right = y_left + 285

area = (x_left, y_left, x_right, y_right)
cropped_img = img.crop(area)
cropped_img.show()

In [ ]:
display(cropped_img)

In [ ]:
img = Image.open("/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_train/data/video-2FTpJaGZLDBCcJfa7-frame-001858-JqymXdTdAivdDToee.jpg")

In [ ]:
display(img)

In [ ]:
x_left = 1452
y_left = 617
x_right = x_left + 41
y_right = y_left + 48

area = (x_left, y_left, x_right, y_right)
cropped_img = img.crop(area)
cropped_img.show()

In [ ]:
display(cropped_img)

In [ ]:
shift = 128
a = shift - 41
b = shift - 48
new_x_left = 1452 - a
new_y_left = 617 - b
new_x_right = x_left + 41 + a
new_y_right = y_left + 48 + b

In [ ]:
area = (new_x_left, new_y_left, new_x_right, new_y_right)
cropped_img = img.crop(area)
cropped_img.show()

In [ ]:
display(cropped_img)

In [ ]:
cropped_img.save('/content/drive/MyDrive/new.jpg')

In [ ]:
!pip install constant

In [ ]:
import constant
constant.x = 1652
constant.y = 1478

In [ ]:
x_left = 84
y_left = 751
size_x = 78
size_y = 59

# Found max example

In [ ]:
f1 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_train/coco.json')
f2 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/coco.json')
f3 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_thermal_train/coco.json')
f4 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_thermal_val/coco.json')
f5 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/video_rgb_test/coco.json')
f6 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/video_thermal_test/coco.json')
data1 = json.load(f1)
data2 = json.load(f2)
data3 = json.load(f3)
data4 = json.load(f4)
data5 = json.load(f5)
data6 = json.load(f6)

def max_coord(file_name):
    x_max = 0
    y_max = 0
    count_1024 = 0
    count = 0
    for i in file_name["annotations"]:
        count += 1
        if i['bbox'][2] > x_max and i['bbox'][2] <= 256:
            x_max = i['bbox'][2]
        if i['bbox'][2] <=256 and i['bbox'][3] <= 256:
            count_1024 += 1
        if i['bbox'][3] > y_max and i['bbox'][3] <= 256:
            y_max = i['bbox'][3]
    print('(x_max:', x_max, '\ty_max:', y_max, ')')
    print('count_256: ', count_1024, '\tcount', count)
    print('-----------------------------')

max_coord(data1)
max_coord(data2)
max_coord(data3)
max_coord(data4)
max_coord(data5)
max_coord(data6)


f1.close()
f2.close()
f3.close()
f4.close()
f5.close()
f6.close()

(x_max: 256 	y_max: 256 )
count_256:  158550 	count 169174
-----------------------------
(x_max: 256 	y_max: 256 )
count_256:  15974 	count 16909
-----------------------------
(x_max: 256 	y_max: 256 )
count_256:  174331 	count 175040
-----------------------------
(x_max: 255 	y_max: 256 )
count_256:  16635 	count 16696
-----------------------------
(x_max: 256 	y_max: 256 )
count_256:  83661 	count 84786
-----------------------------
(x_max: 189 	y_max: 253 )
count_256:  62289 	count 62317
-----------------------------


# OVERRIDE PYCOCOTOOLS

In [ ]:
import json
import time
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
from matplotlib.patches import Polygon
import numpy as np
import copy
import itertools
import pycocotools._mask as maskUtils
# import _mask as maskUtils
import os
from collections import defaultdict
import sys
PYTHON_VERSION = sys.version_info[0]
if PYTHON_VERSION == 2:
    from urllib import urlretrieve
elif PYTHON_VERSION == 3:
    from urllib.request import urlretrieve


def _isArrayLike(obj):
    return hasattr(obj, '__iter__') and hasattr(obj, '__len__')


class my_COCO:
    def __init__(self, annotation_file=None):
        """
        Constructor of Microsoft COCO helper class for reading and visualizing annotations.
        :param annotation_file (str): location of annotation file
        :param image_folder (str): location to the folder that hosts images.
        :return:
        """
        # load dataset
        self.dataset,self.anns,self.cats,self.imgs = dict(),dict(),dict(),dict()
        self.imgToAnns, self.catToImgs = defaultdict(list), defaultdict(list)
        if not annotation_file == None:
            print('loading annotations into memory...')
            tic = time.time()
            dataset = json.load(open(annotation_file, 'r'))
            assert type(dataset)==dict, 'annotation file format {} not supported'.format(type(dataset))
            print('Done (t={:0.2f}s)'.format(time.time()- tic))
            self.dataset = dataset
            self.createIndex()

    def createIndex(self):
        # create index
        print('creating index...')
        anns, cats, imgs = {}, {}, {}
        imgToAnns,catToImgs = defaultdict(list),defaultdict(list)
        if 'annotations' in self.dataset:
            for ann in self.dataset['annotations']:
                imgToAnns[ann['id']].append(ann)
                anns[ann['id']] = ann
                # ???
                imgs[ann['id']] = ann

        # if 'images' in self.dataset:
        #     for img in self.dataset['images']:
        #         imgs[img['id']] = img


        if 'categories' in self.dataset:
            for cat in self.dataset['categories']:
                cats[cat['id']] = cat

        if 'annotations' in self.dataset and 'categories' in self.dataset:
            for ann in self.dataset['annotations']:
                catToImgs[ann['category_id']].append(ann['id'])

        print('index created!')

        # create class members
        self.anns = anns
        self.imgToAnns = imgToAnns
        self.catToImgs = catToImgs
        self.imgs = imgs
        self.cats = cats

    # def info(self):
    #     """
    #     Print information about the annotation file.
    #     :return:
    #     """
    #     for key, value in self.dataset['info'].items():
    #         print('{}: {}'.format(key, value))


# have a question abot imgs(imgIds)
    def getAnnIds(self, imgIds=[], catIds=[], areaRng=[], iscrowd=None):
        """
        Get ann ids that satisfy given filter conditions. default skips that filter
        :param imgIds  (int array)     : get anns for given imgs
               catIds  (int array)     : get anns for given cats
               areaRng (float array)   : get anns for given area range (e.g. [0 inf])
               iscrowd (boolean)       : get anns for given crowd label (False or True)
        :return: ids (int array)       : integer array of ann ids
        """
        imgIds = imgIds if _isArrayLike(imgIds) else [imgIds]
        catIds = catIds if _isArrayLike(catIds) else [catIds]

        if len(imgIds) == len(catIds) == 0:
            anns = self.dataset['annotations']
        else:
            if not len(imgIds) == 0:
                lists = [self.imgToAnns[imgId] for imgId in imgIds if imgId in self.imgToAnns]
                anns = list(itertools.chain.from_iterable(lists))
            else:
                anns = self.dataset['annotations']
            anns = anns if len(catIds)  == 0 else [ann for ann in anns if ann['category_id'] in catIds]
            # anns = anns if len(areaRng) == 0 else [ann for ann in anns if ann['area'] > areaRng[0] and ann['area'] < areaRng[1]]
        if not iscrowd == None:
            ids = [ann['id'] for ann in anns if ann['iscrowd'] == iscrowd]
        else:
            ids = [ann['id'] for ann in anns]
        return ids

    def getCatIds(self, catNms=[], supNms=[], catIds=[]):
        """
        filtering parameters. default skips that filter.
        :param catNms (str array)  : get cats for given cat names
        :param supNms (str array)  : get cats for given supercategory names
        :param catIds (int array)  : get cats for given cat ids
        :return: ids (int array)   : integer array of cat ids
        """
        catNms = catNms if _isArrayLike(catNms) else [catNms]
        # supNms = supNms if _isArrayLike(supNms) else [supNms]
        catIds = catIds if _isArrayLike(catIds) else [catIds]

        if len(catNms) == len(catIds) == 0:
            cats = self.dataset['categories']
        else:
            cats = self.dataset['categories']
            cats = cats if len(catNms) == 0 else [cat for cat in cats if cat['name']          in catNms]
            # cats = cats if len(supNms) == 0 else [cat for cat in cats if cat['supercategory'] in supNms]
            cats = cats if len(catIds) == 0 else [cat for cat in cats if cat['id']            in catIds]
        ids = [cat['id'] for cat in cats]
        return ids

    def getImgIds(self, imgIds=[], catIds=[]):
        '''
        Get img ids that satisfy given filter conditions.
        :param imgIds (int array) : get imgs for given ids
        :param catIds (int array) : get imgs with all given cats
        :return: ids (int array)  : integer array of img ids
        '''
        imgIds = imgIds if _isArrayLike(imgIds) else [imgIds]
        catIds = catIds if _isArrayLike(catIds) else [catIds]

        if len(imgIds) == len(catIds) == 0:
            ids = self.imgs.keys()
        else:
            ids = set(imgIds)
            for i, catId in enumerate(catIds):
                if i == 0 and len(ids) == 0:
                    ids = set(self.catToImgs[catId])
                else:
                    ids &= set(self.catToImgs[catId])
        return list(ids)

    def cls(self, ids=[]):
        target = [0] * 80
        a = ([self.anns[id]['category_id'] for id in ids])
        target[a[0] - 1] = 1
        return target
    
    def loadAnns(self, ids=[]):
        """
        Load anns with the specified ids.
        :param ids (int array)       : integer ids specifying anns
        :return: anns (object array) : loaded ann objects
        """
        if _isArrayLike(ids):
            # list_of_targets = [self.cls(ids)]
            # np_of_targets = np.asarray(list_of_targets)
            # return np_of_targets
            # return [self.cls(ids)]
            return self.cls(ids)
            # return print(type([self.anns[id] for id in ids]))
        elif type(ids) == int:
            # print('int')
            return [self.anns[ids]]

    def loadCats(self, ids=[]):
        """
        Load cats with the specified ids.
        :param ids (int array)       : integer ids specifying cats
        :return: cats (object array) : loaded cat objects
        """
        if _isArrayLike(ids):
            return [self.cats[id] for id in ids]
        elif type(ids) == int:
            return [self.cats[ids]]

    def loadImgs(self, ids=[]):
        """
        Load anns with the specified ids.
        :param ids (int array)       : integer ids specifying img
        :return: imgs (object array) : loaded img objects
        """
        if _isArrayLike(ids):
            return [self.imgs[id] for id in ids]
        elif type(ids) == int:
            return [self.imgs[ids]]

    def download(self, tarDir = None, imgIds = [] ):
        '''
        Download COCO images from mscoco.org server.
        :param tarDir (str): COCO results directory name
               imgIds (list): images to be downloaded
        :return:
        '''
        if tarDir is None:
            print('Please specify target directory')
            return -1
        if len(imgIds) == 0:
            imgs = self.imgs.values()
        else:
            imgs = self.loadImgs(imgIds)
        N = len(imgs)
        if not os.path.exists(tarDir):
            os.makedirs(tarDir)
        for i, img in enumerate(imgs):
            tic = time.time()
            fname = os.path.join(tarDir, img['file_name'])
            if not os.path.exists(fname):
                urlretrieve(img['coco_url'], fname)
            print('downloaded {}/{} images (t={:0.1f}s)'.format(i, N, time.time()- tic))

    def loadNumpyAnnotations(self, data):
        """
        Convert result data from a numpy array [Nx7] where each row contains {imageID,x1,y1,w,h,score,class}
        :param  data (numpy.ndarray)
        :return: annotations (python nested list)
        """
        print('Converting ndarray to lists...')
        assert(type(data) == np.ndarray)
        print(data.shape)
        assert(data.shape[1] == 7)
        N = data.shape[0]
        ann = []
        for i in range(N):
            if i % 1000000 == 0:
                print('{}/{}'.format(i,N))
            ann += [{
                'image_id'  : int(data[i, 0]),
                'bbox'  : [ data[i, 1], data[i, 2], data[i, 3], data[i, 4] ],
                'score' : data[i, 5],
                'category_id': int(data[i, 6]),
                }]
        return ann

    def annToRLE(self, ann):
        """
        Convert annotation which can be polygons, uncompressed RLE to RLE.
        :return: binary mask (numpy 2D array)
        """
        t = self.imgs[ann['image_id']]
        h, w = t['height'], t['width']
        segm = ann['segmentation']
        if type(segm) == list:
            # polygon -- a single object might consist of multiple parts
            # we merge all parts into one mask rle code
            rles = maskUtils.frPyObjects(segm, h, w)
            rle = maskUtils.merge(rles)
        elif type(segm['counts']) == list:
            # uncompressed RLE
            rle = maskUtils.frPyObjects(segm, h, w)
        else:
            # rle
            rle = ann['segmentation']
        return rle

    def annToMask(self, ann):
        """
        Convert annotation which can be polygons, uncompressed RLE, or RLE to binary mask.
        :return: binary mask (numpy 2D array)
        """
        rle = self.annToRLE(ann)
        m = maskUtils.decode(rle)
        return m
    


In [ ]:
class my_CocoDetection(VisionDataset):
    """`MS Coco Detection <https://cocodataset.org/#detection-2016>`_ Dataset.

    It requires the `COCO API to be installed <https://github.com/pdollar/coco/tree/master/PythonAPI>`_.

    Args:
        root (string): Root directory where images are downloaded to.
        annFile (string): Path to json annotation file.
        transform (callable, optional): A function/transform that  takes in an PIL image
            and returns a transformed version. E.g, ``transforms.ToTensor``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        transforms (callable, optional): A function/transform that takes input sample and its target as entry
            and returns a transformed version.
    """

    def __init__(
        self,
        root: str,
        annFile: str,
        transform: Optional[Callable] = None,
        target_transform: Optional[Callable] = None,
        transforms: Optional[Callable] = None,
    ) -> None:
        super().__init__(root, transforms, transform, target_transform)
        from pycocotools.coco import COCO

        self.coco = my_COCO(annFile)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def _load_image(self, id: int) -> Image.Image:
        path = self.coco.loadImgs(id)[0]["file_name"]
        # im = Image.open(os.path.join(self.root, os.path.basename(path))).convert("RGB")
        # display(im)
        return Image.open(os.path.join(self.root, os.path.basename(path)))

    def _load_target(self, id: int):
        # target = self.coco.loadAnns(self.coco.getAnnIds(id))
        # if self.target_transform is not None:
        #   target = self.target_transform(target)
        # return target
        return self.coco.loadAnns(self.coco.getAnnIds(id))

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        id = self.ids[index]
        image = self._load_image(id)
        # target = self._load_target(id)
        target = torch.as_tensor(self._load_target(id), dtype = torch.float32)
        # print(type(image))
        if self.transforms is not None:
            image, target = self.transforms(image, target)
        return image, target

    def __len__(self) -> int:
        return len(self.ids)
    
    def printf(self):
      print(self.root)


In [ ]:
transform = Compose([PILToTensor(), ConvertImageDtype(torch.float)])

In [ ]:
path2data="/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/new_data"
path2json="/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/ann.json"

In [ ]:
# coco = my_CocoDetection(path2data, path2json, transform = PILToTensor())
coco = my_CocoDetection(path2data, path2json, transform = transform)
# coco = my_CocoDetection(path2data, path2json, transforms=ToTensor())

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!


In [ ]:
coco

Dataset my_CocoDetection
    Number of datapoints: 15974
    Root location: /content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/new_data
    StandardTransform
Transform: Compose(
               PILToTensor()
               ConvertImageDtype()
           )

In [ ]:
coco.__getitem__(58)

(tensor([[[0.5216, 0.5137, 0.5020,  ..., 0.5529, 0.5490, 0.5490],
          [0.5020, 0.4980, 0.4863,  ..., 0.5333, 0.5373, 0.5412],
          [0.4941, 0.4902, 0.4824,  ..., 0.5137, 0.5255, 0.5294],
          ...,
          [0.8039, 0.8039, 0.8039,  ..., 0.7843, 0.7843, 0.7843],
          [0.7922, 0.7922, 0.7922,  ..., 0.7843, 0.7843, 0.7843],
          [0.7882, 0.7882, 0.7882,  ..., 0.7882, 0.7843, 0.7843]],
 
         [[0.5333, 0.5216, 0.5098,  ..., 0.5020, 0.4980, 0.4980],
          [0.5098, 0.5059, 0.4941,  ..., 0.4941, 0.4980, 0.5020],
          [0.5020, 0.4980, 0.4902,  ..., 0.4824, 0.4941, 0.4980],
          ...,
          [0.8118, 0.8118, 0.8118,  ..., 0.7843, 0.7843, 0.7843],
          [0.8078, 0.8078, 0.8078,  ..., 0.7843, 0.7843, 0.7843],
          [0.8039, 0.8039, 0.8039,  ..., 0.7882, 0.7843, 0.7843]],
 
         [[0.4667, 0.4667, 0.4588,  ..., 0.4667, 0.4627, 0.4627],
          [0.4549, 0.4510, 0.4431,  ..., 0.4549, 0.4588, 0.4627],
          [0.4471, 0.4431, 0.4392,  ...,

In [ ]:
coco._load_target(12)

[0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [ ]:
batch_size = 3

In [ ]:
coco_loader = DataLoader(coco, batch_size, shuffle=True, num_workers = 0)

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
import torchvision.datasets as dset

In [ ]:
device = get_default_device()
device

device(type='cpu')

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    # rewrite
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        labels1 = labels.softmax(dim=1)
        # print(labels, labels.size())
        print(labels)
        print(out)
        # print (out, out.size())
        loss = F.cross_entropy(out, labels1)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
      
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [ ]:
resnet34 = models.resnet34()
resnet34

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class FLIR80Resnet(ImageClassificationBase):
        def __init__(self):
                super().__init__()
                self.network = torchvision.models.resnet34()
                num_ftrs = self.network.fc.in_features
                self.network.fc = nn.Linear(num_ftrs, 80)
        
        def forward(self, xb):
                return self.network(xb)
            
model = FLIR80Resnet()
model

FLIR80Resnet(
  (network): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [ ]:
model = to_device(FLIR80Resnet(), device)

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, 
        weight_decay=0, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr, weight_decay=weight_decay)
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
history = [evaluate(model, coco_loader)]
history

tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 

RuntimeError: ignored

In [ ]:
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
loss = F.cross_entropy(input, target)
loss.backward()


tensor([[ 0.6092,  1.5785,  0.3189,  0.5635, -0.0618],
        [-1.0746, -1.8930,  1.2565, -0.7404, -0.8455],
        [-0.9029, -0.0176,  0.2300, -0.2433, -0.1535]], requires_grad=True)
tensor([[0.0612, 0.2935, 0.2335, 0.0501, 0.3618],
        [0.0497, 0.0884, 0.6734, 0.0731, 0.1155],
        [0.4664, 0.2580, 0.0399, 0.0979, 0.1377]])


In [ ]:
path2data="/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/data"
path2json="/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/coco.json"

In [ ]:
coco_train_detect = my_CocoDetection(path2data, path2json, transform=PILToTensor())

loading annotations into memory...
Done (t=2.26s)
creating index...
index created!


In [ ]:
# return all targets from image_id (arguments = 'image_id')
coco_train_detect.__getitem__(1)

In [ ]:
coco_train_detect._load_target(1)

[{'area': 110,
  'bbox': [360, 217, 10, 11],
  'category_id': 12,
  'extra_info': {'human_annotated': 'human', 'occluded': 'no_(fully_visible)'},
  'id': 16,
  'image_id': 1,
  'iscrowd': False,
  'segmentation': [[360, 217, 370, 217, 360, 228, 370, 228]]},
 {'area': 189,
  'bbox': [199, 223, 21, 9],
  'category_id': 12,
  'extra_info': {'human_annotated': 'human',
   'occluded': '1%_-_70%_occluded_(partially_occluded)'},
  'id': 17,
  'image_id': 1,
  'iscrowd': False,
  'segmentation': [[199, 223, 220, 223, 199, 232, 220, 232]]},
 {'area': 602,
  'bbox': [489, 222, 14, 43],
  'category_id': 1,
  'extra_info': {'human_annotated': 'human',
   'occluded': 'no_(fully_visible)',
   'specify_if_human_or_not': 'human'},
  'id': 18,
  'image_id': 1,
  'iscrowd': False,
  'segmentation': [[489, 222, 503, 222, 489, 265, 503, 265]]},
 {'area': 748,
  'bbox': [421, 245, 34, 22],
  'category_id': 2,
  'extra_info': {'human_annotated': 'human',
   'occluded': '1%_-_70%_occluded_(partially_occluded

# Create new catalog with objects 

In [ ]:
def max_coord(file_name):
    x_max = 0
    y_max = 0
    count_1024 = 0
    count = 0
    for i in file_name["annotations"]:
        count += 1
        if i['bbox'][2] > x_max and i['bbox'][2] <= 256:
            x_max = i['bbox'][2]
        if i['bbox'][2] <=256 and i['bbox'][3] <= 256:
            count_1024 += 1
        if i['bbox'][3] > y_max and i['bbox'][3] <= 256:
            y_max = i['bbox'][3]
    print('(x_max:', x_max, '\ty_max:', y_max, ')')
    print('count_256: ', count_1024, '\tcount', count)
    print('-----------------------------')


(x_max: 256 	y_max: 256 )
count_256:  158550 	count 169174
-----------------------------
(x_max: 256 	y_max: 256 )
count_256:  15974 	count 16909
-----------------------------
(x_max: 256 	y_max: 256 )
count_256:  174331 	count 175040
-----------------------------
(x_max: 255 	y_max: 256 )
count_256:  16635 	count 16696
-----------------------------
(x_max: 256 	y_max: 256 )
count_256:  83661 	count 84786
-----------------------------
(x_max: 189 	y_max: 253 )
count_256:  62289 	count 62317
-----------------------------


Old code without overriding annotation

In [ ]:
def crop_img(path, file_name):
  shift = 256
  x_left = 0
  y_left = 0
  x_right = 0
  y_right = 0
  for i in file_name["annotations"]:
    if (i['bbox'][2] <= shift and i['bbox'][3] <= shift):
      if i['bbox'][2] % 2 == 1:
        i['bbox'][2] += 1
      if i['bbox'][3] % 2 == 1:
        i['bbox'][3] += 1
      a = int((shift - i['bbox'][2])/2)
      b = int((shift - i['bbox'][3])/2)
      x_left = i['bbox'][0] - a
      y_left = i['bbox'][1] - b
      x_right = i['bbox'][0] + i['bbox'][2] + a
      y_right = i['bbox'][1] + i['bbox'][3] + b
      for j in file_name["images"]:
        if i['image_id'] == j['id']:
          img = Image.open(path + j['file_name'])
          area = (x_left, y_left, x_right, y_right)
          cropped_img = img.crop(area)
          cropped_img.save(str(path + 'new_data/' + (os.path.splitext(os.path.basename(j['file_name'])))[0] + '_obj' + str(i['id']) + '.jpg'))
          break

New code with overriding annotation

In [ ]:
# def write_json(new_data, new_data2, new_data3, new_data4, new_data5, new_data6,  filename = 'ann.json'):
def write_json_ann(new_data, new_data2, new_data3, new_data4, new_data5, new_data6,  path):
    filename = path + 'ann.json'
    with open(filename, 'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        # Join new_data with file_data inside annotation
        new = {new_data: new_data2, new_data3: new_data4, new_data5: new_data6}
        file_data["annotations"].append(new)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file, indent = 4)

In [ ]:
# def write_json(new_data, new_data2, new_data3, new_data4, filename = 'ann.json'):
def write_json_cat(new_data, new_data2, new_data3, new_data4, path):
    filename = path + 'ann.json'
    with open(filename, 'r+') as file:
        file_data = json.load(file)
        new = {new_data: new_data2, new_data3: new_data4}
        file_data["categories"].append(new)
        file.seek(0)
        json.dump(file_data, file, indent = 4)

In [ ]:
def crop_img_with_ann(path, file_name):
  shift = 256
  x_left = 0
  y_left = 0
  x_right = 0
  y_right = 0
  for i in file_name["annotations"]:
    if (i['bbox'][2] <= 256 and i['bbox'][3] <= 256):
      if i['bbox'][2] % 2 == 1:
        i['bbox'][2] += 1
      if i['bbox'][3] % 2 == 1:
        i['bbox'][3] += 1
      a = int((shift - i['bbox'][2])/2)
      b = int((shift - i['bbox'][3])/2)
      x_left = i['bbox'][0] - a
      y_left = i['bbox'][1] - b
      x_right = i['bbox'][0] + i['bbox'][2] + a
      y_right = i['bbox'][1] + i['bbox'][3] + b
      for j in file_name["images"]:
        if i['image_id'] == j['id']:
          img = Image.open(path + j['file_name'])
          area = (x_left, y_left, x_right, y_right)
          cropped_img = img.crop(area)
          cropped_img.save(str(path + 'new_data/' + (os.path.splitext(os.path.basename(j['file_name'])))[0] + '_obj' + str(i['id']) + '.jpg'))
          write_json_ann("id", i['id'], "category_id", i['category_id'], "file_name", str(path + 'new_data/' + (os.path.splitext(os.path.basename(j['file_name'])))[0] + '_obj' + str(i['id']) + '.jpg'), path)
          break
  for i in file_name["categories"]:
    write_json_cat("id", i['id'], "name", i['name'], path)

In [ ]:
path = '/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_train/'
f1 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_train/coco.json')
f2 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/coco.json')
f3 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_thermal_train/coco.json')
f4 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_thermal_val/coco.json')
f5 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/video_rgb_test/coco.json')
f6 = open('/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/video_thermal_test/coco.json')

data1 = json.load(f1)
data2 = json.load(f2)
data3 = json.load(f3)
data4 = json.load(f4)
data5 = json.load(f5)
data6 = json.load(f6)

max_coord(data1)
max_coord(data2)
max_coord(data3)
max_coord(data4)
max_coord(data5)
max_coord(data6)

f1.close()
f2.close()
f3.close()
f4.close()
f5.close()
f6.close()

In [ ]:
path = '/content/drive/MyDrive/FLIR_ADAS_v2/FLIR_ADAS_v2/images_rgb_val/'
f1 = open(path + 'coco.json')
data1 = json.load(f1)
crop_img(path, data1)
f1.close()